In [ ]:
import pandas as pd
import os
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout,Flatten,Activation
from keras.layers.convolutional import Conv2D,MaxPooling2D,ZeroPadding2D,AveragePooling2D
from keras.utils import np_utils
import cv2
import skimage
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint

In [ ]:
def get_data(start,end):
  print("Reading Files")
  x=[]
  y=[]
  vv=-1
  d=0
  for f in os.listdir("asl_alphabet_train"):
    if not f.startswith('.' and 'm'):
      if f.startswith('p'):
        continue
      vv=vv+1
      if vv%3==0:
        print("Files Completed:")
        print(vv)
      if f in ['A']:
        label=0
      elif f in ['B']:
        label=1
      elif f in ['C']:
        label=2
      elif f in ['D']:
        label=3
      elif f in ['E']:
        label=4
      elif f in ['F']:
        label=5
      elif f in ['G']:
        label=6
      elif f in ['H']:
        label=7
      elif f in ['I']:
        label=8
      elif f in ['J']:
        label=9
      elif f in ['K']:
        label=10
      elif f in ['L']:
        label=11
      elif f in ['M']:
        label=12
      elif f in ['N']:
        label=13
      elif f in ['O']:
        label=14
      elif f in ['P']:
        label=15
      elif f in ['Q']:
        label=16
      elif f in ['R']:
        label=17
      elif f in ['S']:
        label=18
      elif f in ['T']:
        label=19
      elif f in ['U']:
        label=20
      elif f in ['V']:
        label=21
      elif f in ['W']:
        label=22
      elif f in ['X']:
        label=23
      elif f in ['Y']:
        label=24
      elif f in ['Z']:
        label=25
      elif f in ['del']:
        label=26
      elif f in ['nothing']:
        label=27
      elif f in ['space']:
        label=28
      else:
        label=29
      s1=-1
      for image in os.listdir('./asl_alphabet_train/'+f):
        d=d+1
        s1+=1
        if s1<start:
          continue
        if s1>end:
          break
        img_file=cv2.imread('./asl_alphabet_train/'+f+'/'+image)
        #img_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2GRAY)
        img_file1=cv2.cvtColor(img_file, cv2.COLOR_BGR2HSV)
        lower = np.array([0,40,12])
        upper = np.array([190,202,220])
        mask = cv2.inRange(img_file1, lower, upper)
        res = cv2.bitwise_and(img_file,img_file, mask= mask)
        mask1=cv2.bitwise_not(res)
        res1 = cv2.bitwise_or(mask1,mask1, mask= mask)
        img_file=cv2.bitwise_not(res1)
        img_file=cv2.cvtColor(img_file,cv2.COLOR_BGR2GRAY)        
        k=np.ones((2,1),np.uint8)
        img_file=cv2.erode(img_file,k,iterations=2)
        q12=img_file
        #img_file=sobel_converter(img_file)
        if img_file is not None:
            
            img_file = cv2.resize(img_file,(50,50),interpolation=cv2.INTER_AREA)
            img_arr = np.asarray(img_file)
            x.append(img_arr)
            y.append(label)
            #cv2.imwrite(os.path.join('sample','test '+str(d)+'.jpg'),q12)
            
  x=np.asarray(x)
  y=np.asarray(y)
  print("Reading Completed")
  x=x.reshape(x.shape[0],x.shape[1],x.shape[2],1)
  return x,y

In [ ]:
x,y=get_data(0,150)
y=np_utils.to_categorical(y,num_classes=30)
print(y)
x,y= shuffle(x,y,random_state=13)

In [ ]:
def get_model():
    model=Sequential()
    model.add(Conv2D(32,(2,2),padding='same',input_shape=(50,50,1)))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(2,2),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128,(2,2),padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(256,(2,2),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))
    model.add(Conv2D(512,(2,2),padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(512,(2,2),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(30,activation='softmax'))  
    return model


In [ ]:
model=get_model()

In [ ]:
x.shape

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

filepath="model4.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(x,y,epochs=10,batch_size=32,verbose=1,callbacks=callbacks_list)
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
new_model=load_model("model4.h5")
for i in range(1,10):
    print(i)
    start=i*250
    end=(i+1)*250
    x,y=get_data(start,end)
    y=np_utils.to_categorical(y,num_classes=30)
    x,y= shuffle(x,y,random_state=13)
    new_model = load_model("model4.h5")
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    new_model.fit(x,y,epochs=5,batch_size=32, callbacks=callbacks_list)
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['loss'])

plt.show()